In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Obtain the imdb review dataset from tensorflow datasets
dataset = tfds.load('imdb_reviews', as_supervised=True)

# Seperate test and train datasets
train_dataset, test_dataset = dataset['train'], dataset['test']

# Split the test and train data into batches of 32
# and shuffling the training set
batch_size = 32
train_dataset = train_dataset.shuffle(10000)
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\suzan.awinat\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete0XFH56\imdb_review…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\suzan.awinat\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete0XFH56\imdb_review…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\suzan.awinat\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete0XFH56\imdb_review…

Dataset imdb_reviews downloaded and prepared to C:\Users\suzan.awinat\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [3]:
#iter function to create an iterator for the training dataset and then extracts the first batch
example, label = next(iter(train_dataset))
print('Text:\n', example.numpy()[0])
print('\nLabel: ', label.numpy()[0])


Text:
 b"Wow. This was probably the worst DCOM ever. I watched the first half hour and I laughed. Brenda Song plays Wendy, the popular girl with the hot jock boyfriend and stuck up friends who is determined to be Homecoming Queen. She is supposed to save the world as a warrior, and Shin comes to her aid to help her with her Martial Arts. Shin teaches her the skills of a snake, tiger, etc. and she has to learn certain techniques to save the world.<br /><br />This movie is great for kids who want to learn about Martial Arts and the Chinese culture but the acting and casting was horrible.<br /><br />Brenda Song is a comedic actress and I can't see her playing a serious role. It was laugh out loud funny watching her cry over Shin. Shin couldn't act at all, and everything was totally unbelievable.<br /><br />I watched this movie and tried to think of something similar, and the thing I came up with was the Power Rangers. This movie is so fake and the stunts were so Power Ranger-esquire that 

In [4]:
# Using the TextVectorization layer to normalize, split, and map strings
# to integers.
encoder = tf.keras.layers.TextVectorization(max_tokens=10000)
encoder.adapt(train_dataset.map(lambda text, _: text))

# Extracting the vocabulary from the TextVectorization layer.
vocabulary = np.array(encoder.get_vocabulary())

# Encoding a test example and decoding it back.
original_text = example.numpy()[0]
encoded_text = encoder(original_text).numpy()
decoded_text = ' '.join(vocabulary[encoded_text])

print('original: ', original_text)
print('encoded: ', encoded_text)
print('decoded: ', decoded_text)

original:  b"Wow. This was probably the worst DCOM ever. I watched the first half hour and I laughed. Brenda Song plays Wendy, the popular girl with the hot jock boyfriend and stuck up friends who is determined to be Homecoming Queen. She is supposed to save the world as a warrior, and Shin comes to her aid to help her with her Martial Arts. Shin teaches her the skills of a snake, tiger, etc. and she has to learn certain techniques to save the world.<br /><br />This movie is great for kids who want to learn about Martial Arts and the Chinese culture but the acting and casting was horrible.<br /><br />Brenda Song is a comedic actress and I can't see her playing a serious role. It was laugh out loud funny watching her cry over Shin. Shin couldn't act at all, and everything was totally unbelievable.<br /><br />I watched this movie and tried to think of something similar, and the thing I came up with was the Power Rangers. This movie is so fake and the stunts were so Power Ranger-esquire t

In [5]:
# Creating the model
model = tf.keras.Sequential([
	encoder,
	tf.keras.layers.Embedding(
		len(encoder.get_vocabulary()), 64, mask_zero=True),
	tf.keras.layers.Bidirectional(
		tf.keras.layers.LSTM(64, return_sequences=True)),
	tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
	tf.keras.layers.Dense(64, activation='relu'),
	tf.keras.layers.Dense(1)
])

# Summary of the model
model.summary()

# Compile the model
model.compile(
	loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
	optimizer=tf.keras.optimizers.Adam(),
	metrics=['accuracy']
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, None, 64)          640000    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [ ]:
# Training the model and validating it on test set
history = model.fit(
	train_dataset, 
	epochs=5,
	validation_data=test_dataset,
)

Epoch 1/5



782/782 [==============================] - 1443s 2s/step - loss: 0.4313 - accuracy: 0.7902 - val_loss: 0.3491 - val_accuracy: 0.8551
Epoch 2/5
782/782 [==============================] - 1588s 2s/step - loss: 0.2711 - accuracy: 0.8897 - val_loss: 0.3215 - val_accuracy: 0.8736
Epoch 3/5
782/782 [==============================] - 1887s 2s/step - loss: 0.1831 - accuracy: 0.9301 - val_loss: 0.3352 - val_accuracy: 0.8762
Epoch 4/5
782/782 [==============================] - 2256s 3s/step - loss: 0.1294 - accuracy: 0.9527 - val_loss: 0.3771 - val_accuracy: 0.8669
Epoch 5/5
156/782 [====>.........................] - ETA: 25:30 - loss: 0.0579 - accuracy: 0.9818

In [ ]:
# Plotting the accuracy and loss over time

# Training history
history_dict = history.history

# Seperating validation and training accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# Seperating validation and training loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# Plotting
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.plot(acc)
plt.plot(val_acc)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Accuracy', 'Validation Accuracy'])

plt.subplot(1, 2, 2)
plt.plot(loss)
plt.plot(val_loss)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Loss', 'Validation Loss'])

plt.show()


In [ ]:
# Making predictions
sample_text = (
	'''The movie was so good and the animation are so dope. 
	I would recommend my friends to watch it.'''
)
predictions = model.predict(np.array([sample_text]))
print(*predictions[0])

# Print the label based on the prediction
if predictions[0] > 0:
	print('The review is positive')
else:
	print('The review is negative')
